In [ ]:
import pandas as pd
import plotly.graph_objects as go

import torch.utils.data as data_utils

import warnings
warnings.filterwarnings('ignore')

from preprocessing import *
from postprocessing import *

In [ ]:
model_type = "lstm"

if model_type == "linear_ae":
    from linear_ae import *
elif model_type == "conv_ae":
    from convolutional_ae import *
elif model_type == "lstm_ae":
    from lstm_ae import *
elif model_type == "transformer":
    from transformer import *
elif model_type == "lstm":
    from lstm import *

if torch.cuda.is_available():
    device =  torch.device('cuda')
else:
    device = torch.device('cpu')

In [ ]:
n_channels = 1
train_window = 72
w_size = train_window * n_channels
BATCH_SIZE = 128

In [ ]:
if model_type == "lstm_ae" or model_type == "conv_ae" or model_type == "lstm":
    z_size = 32
d_model = 64
dim_ff = 256
n_layer = 3
n_head = 4
# Create the model and send it on the gpu device
if model_type == "lstm_ae":
    model = LstmAE(n_channels, z_size, train_window)
elif model_type == "conv_ae":
    model = ConvAE(n_channels, z_size)
elif model_type == "linear_ae":
    model = LinearAE(w_size, z_size)
elif model_type == "transformer":
    model = Transformer(n_channels, d_model, dim_ff, n_layer, train_window, n_head)
elif model_type == "lstm":
    model = LstmModel(n_channels, z_size, 1)

model = model.to(device)
print(model)

In [ ]:
# Recover checkpoint
checkpoint_dir = "/content/drive/MyDrive/Prova_Transformers_production_ad/production_big_linear_ae_200_03_01.pth"
checkpoint = torch.load(checkpoint_dir) # add parameter map_location='cpu', if you are not able to switch to gpu for testing
if model_type == "transformer" or model_type == "lstm":
    model.load_state_dict(checkpoint)
else:
    model.encoder.load_state_dict(checkpoint['encoder'])
    model.decoder.load_state_dict(checkpoint['decoder'])

## Inference on DATACELLAR data

In [ ]:
dc = pd.read_csv("/content/drive/MyDrive/Prova_Transformers_production_ad/ad_production_dc.csv")
dc

,user_id,local_date_str,datetime,type,generation_kwh,nominal_power_w
0,ES0184000000051513CQ0FA000,2023-05-22,2023-05-21T22:00:00,b11,0.00,20000
1,ES0184000000051513CQ0FA000,2023-05-22,2023-05-21T23:00:00,b11,0.00,20000
2,ES0184000000051513CQ0FA000,2023-05-22,2023-05-22T00:00:00,b11,0.00,20000
3,ES0184000000051513CQ0FA000,2023-05-22,2023-05-22T01:00:00,b11,0.00,20000
4,ES0184000000051513CQ0FA000,2023-05-22,2023-05-22T02:00:00,b11,0.00,20000
...,...,...,...,...,...,...
4995,ES0184000000051513CQ0FA000,2023-12-18,2023-12-18T13:00:00,b11,15.08,20000
4996,ES0184000000051513CQ0FA000,2023-12-18,2023-12-18T14:00:00,b11,14.90,20000
4997,ES0184000000051513CQ0FA000,2023-12-18,2023-12-18T15:00:00,b11,13.18,20000
4998,ES0184000000051513CQ0FA000,2023-12-18,2023-12-18T16:00:00,b11,9.15,20000


In [ ]:
X_test = dc[['generation_kwh']]

In [ ]:
if model_type == "transformer" or model_type == "lstm":
  x_test, y_test = create_transformer_sequences(X_test, train_window)
else:
  test_seq = create_sequences(X_test, train_window, train_window)

In [ ]:
if model_type == "transformer" or model_type == "lstm":
  test_loader = torch.utils.data.DataLoader(data_utils.TensorDataset(torch.from_numpy(x_test).float(), torch.from_numpy(y_test).float()), batch_size = BATCH_SIZE, shuffle = False, num_workers = 0)
elif model_type == "linear_ae":
  test_loader = torch.utils.data.DataLoader(data_utils.TensorDataset(
    torch.from_numpy(test_seq).float().view(([test_seq.shape[0], w_size]))
) , batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
else:
  test_loader = torch.utils.data.DataLoader(data_utils.TensorDataset(
    torch.from_numpy(test_seq).float()
) , batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

In [ ]:
results, w = testing(model, test_loader, device)

In [ ]:
if model_type == "transformer" or model_type == "lstm":
  r = np.concatenate([torch.stack(w[:-1]).flatten().detach().cpu().numpy(), w[-1].flatten().detach().cpu().numpy()])
else:
  r = w[0].flatten().detach().cpu().numpy()

In [ ]:
dim_test = test_seq.shape[0] * test_seq.shape[1]

In [ ]:
if model_type == "transformer" or model_type == "lstm":
    df_t = get_transformer_dataset(y_test, r, X_test, train_window)
else:
    df_t = get_predicted_dataset(pd.DataFrame(X_test[:dim_test], columns=['generation_kwh']), r)

### LSTM Autoencoder

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_t.index, y=df_t['generation_kwh'], name='production_test'))
fig.add_trace(go.Scatter(x=df_t.index, y=df_t['reconstruction'], name='reconstruction'))
fig.update_layout(showlegend=True, title='Energy Production')
fig.show()

In [ ]:
threshold_method = 0
percentile = 97.5
weight_overall = 0.5
k = 1.5

df_t = anomaly_detection(df_t, df_t, threshold_method, percentile, weight_overall, k)

In [ ]:
df_t

,generation_kwh,reconstruction,abs_loss,rel_loss,threshold,predicted_anomaly
0,0.000000,0.080312,0.080312,1.000000,0.471122,0
1,0.000000,0.053876,0.053876,1.000000,0.471122,0
2,0.000000,0.103023,0.103023,1.000000,0.471122,0
3,0.000000,0.078287,0.078287,1.000000,0.471122,0
4,0.000000,0.067878,0.067878,1.000000,0.471122,0
...,...,...,...,...,...,...
4963,0.000000,-0.001935,0.001935,1.000000,0.471122,0
4964,0.000000,-0.003378,0.003378,1.000000,0.471122,0
4965,0.000000,0.000490,0.000490,1.000000,0.471122,0
4966,0.000513,-0.000071,0.000584,8.217097,0.471122,0


In [ ]:
df_t.predicted_anomaly.sum() / len(df_t)

0.025161030595813205

In [ ]:
predicted_anomalies = df_t[df_t.predicted_anomaly == 1]

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_t.index, y=df_t['generation_kwh'], name='production_test'))
fig.add_trace(go.Scatter(x=df_t.index, y=df_t['reconstruction'], name='reconstruction'))
fig.add_trace(go.Scatter(x=predicted_anomalies.index, y=predicted_anomalies['generation_kwh'], mode='markers', marker=dict(color='green'), name='Predicted_Anomaly'))
fig.update_layout(showlegend=True, title='Energy Production')
fig.show()

### CONV Autoencoder

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_t.index, y=df_t['generation_kwh'], name='production_test'))
fig.add_trace(go.Scatter(x=df_t.index, y=df_t['reconstruction'], name='reconstruction'))
fig.update_layout(showlegend=True, title='Energy Production')
fig.show()

In [ ]:
threshold_method = 0
percentile = 97.5
weight_overall = 0.5
k = 1.5

df_t = anomaly_detection(df_t, df_t, threshold_method, percentile, weight_overall, k)

In [ ]:
df_t.predicted_anomaly.sum() / len(df_t)

0.025161030595813205

In [ ]:
predicted_anomalies = df_t[df_t.predicted_anomaly == 1]

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_t.index, y=df_t['generation_kwh'], name='production_test'))
fig.add_trace(go.Scatter(x=df_t.index, y=df_t['reconstruction'], name='reconstruction'))
fig.add_trace(go.Scatter(x=predicted_anomalies.index, y=predicted_anomalies['generation_kwh'], mode='markers', marker=dict(color='green'), name='Predicted_Anomaly'))
fig.update_layout(showlegend=True, title='Energy Production')
fig.show()

### Linear Autoencoder

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_t.index, y=df_t['generation_kwh'], name='production_test'))
fig.add_trace(go.Scatter(x=df_t.index, y=df_t['reconstruction'], name='reconstruction'))
fig.update_layout(showlegend=True, title='Energy Production')
fig.show()

In [ ]:
threshold_method = 0
percentile = 97.50
weight_overall = 0.5
k = 1.5

df_t = anomaly_detection(df_t, df_t, threshold_method, percentile, weight_overall, k)

In [ ]:
df_t.predicted_anomaly.sum() / len(df_t)

0.025161030595813205

In [ ]:
predicted_anomalies = df_t[df_t.predicted_anomaly == 1]

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_t.index, y=df_t['generation_kwh'], name='production_test'))
fig.add_trace(go.Scatter(x=df_t.index, y=df_t['reconstruction'], name='reconstruction'))
fig.add_trace(go.Scatter(x=predicted_anomalies.index, y=predicted_anomalies['generation_kwh'], mode='markers', marker=dict(color='green'), name='Predicted_Anomaly'))
fig.update_layout(showlegend=True, title='Energy Production')
fig.show()

### Transformer

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_t.index, y=df_t['generation_kwh'], name='production_test'))
fig.add_trace(go.Scatter(x=df_t.index, y=df_t['reconstruction'], name='reconstruction'))
fig.update_layout(showlegend=True, title='Energy Production')
fig.show()

In [ ]:
threshold_method = 0
percentile = 97.5
weight_overall = 0.5
k = 1.5

df_t = anomaly_detection(df_t, df_t, threshold_method, percentile, weight_overall, k)

In [ ]:
df_t.predicted_anomaly.sum() / len(df_t)

0.025162337662337664

In [ ]:
predicted_anomalies = df_t[df_t.predicted_anomaly == 1]

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_t.index, y=df_t['generation_kwh'], name='production_test'))
fig.add_trace(go.Scatter(x=df_t.index, y=df_t['reconstruction'], name='reconstruction'))
fig.add_trace(go.Scatter(x=predicted_anomalies.index, y=predicted_anomalies['generation_kwh'], mode='markers', marker=dict(color='green'), name='Predicted_Anomaly'))
fig.update_layout(showlegend=True, title='Energy Production')
fig.show()

### LSTM (Forecasting)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_t.index, y=df_t['generation_kwh'], name='production_test'))
fig.add_trace(go.Scatter(x=df_t.index, y=df_t['reconstruction'], name='reconstruction'))
fig.update_layout(showlegend=True, title='Energy Production')
fig.show()

In [ ]:
threshold_method = 0
percentile = 97.5
weight_overall = 0.5
k = 1.5

df_t = anomaly_detection(df_t, df_t, threshold_method, percentile, weight_overall, k)

In [ ]:
df_t.predicted_anomaly.sum() / len(df_t)

0.025162337662337664

In [ ]:
predicted_anomalies = df_t[df_t.predicted_anomaly == 1]

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_t.index, y=df_t['generation_kwh'], name='production_test'))
fig.add_trace(go.Scatter(x=df_t.index, y=df_t['reconstruction'], name='reconstruction'))
fig.add_trace(go.Scatter(x=predicted_anomalies.index, y=predicted_anomalies['generation_kwh'], mode='markers', marker=dict(color='green'), name='Predicted_Anomaly'))
fig.update_layout(showlegend=True, title='Energy Production')
fig.show()